In [27]:
from datetime import datetime
import time
import os
from tqdm import tqdm
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.corpus import stopwords
import pyLDAvis.gensim_models
import jieba.posseg as jp,jieba
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib
def extract_info(x):
    CreationDate = re.findall(r'CreationDate = (.*?)//',x)[0]

    Title = re.findall(r'Title = (.*?)//',x)[0]

    Tags = re.findall(r'Tags = (.*?)//',x)[0]
    Tags = Tags.split('><')
    Tags[0] = Tags[0].lstrip('<')
    Tags[-1] = Tags[-1].rstrip('>')
    Tags = ' '.join(Tags)

    Score = re.findall(r'Score = (.*?)//',x)[0]


    AnswerCount = re.findall(r'AnswerCount = (.*?)//',x)
    if AnswerCount==[] or AnswerCount==['']:
        AnswerCount = re.findall(r'AnswerCount = (.*?)\n',x)
        if AnswerCount == [] or AnswerCount==['']:
            AnswerCount = None
        else:
            AnswerCount = AnswerCount[0]
    else:
        AnswerCount = AnswerCount[0]


    CommentCount = re.findall(r'CommentCount = (.*?)//',x)
    if CommentCount==[] or CommentCount==['']:
        CommentCount = re.findall(r'CommentCount = (.*?)\n',x)
        if CommentCount==[] or CommentCount==['']:
            CommentCount = None
        else:
            CommentCount = CommentCount[0]
    else:
        CommentCount = CommentCount[0]

    FavoriteCount = re.findall(r'FavoriteCount = (.*?)//',x)
    if FavoriteCount==[] or FavoriteCount==['']:
        FavoriteCount = re.findall(r'FavoriteCount = (.*?)\n',x)
        if FavoriteCount==[] or FavoriteCount==['']:
            FavoriteCount = None
        else:
            FavoriteCount = FavoriteCount[0]
    else:
        FavoriteCount = FavoriteCount[0]



    return CreationDate, Title, Tags, Score, AnswerCount, CommentCount, FavoriteCount

In [28]:
path1 = 'D:/CCFinder/Commit2So(A commit will correspond to multiple sos)/'
list1 = os.listdir(path1)
len(list1),list1[0]


with open('./commit_name.txt','w', encoding='utf-8') as f:
    for i in list1:
        f.write(i)
        f.write('\n')


In [29]:
from nltk.corpus import stopwords
import nltk
from gensim.models import word2vec
w2v_model = word2vec.Word2Vec.load('./word2vec.model')

def mean_vector(words):
    vec = np.zeros(300).reshape((1, 300))
    for word in words:
        try:
            vec += w2v_model[word].reshape((1, 300))
        except KeyError:
            continue
    return_vec = vec/len(words)
    return_vec = return_vec.reshape((300,))
    return return_vec.tolist()


# def max_vector(words):
#     vec = np.zeros(300).reshape((1, 300))
#     for word in words:
#         try:
#             vec = np.concatenate((vec, w2v_model[word].reshape((1, 300))),axis=0)
#         except KeyError:
#             continue
#     return_vec = vec.max(axis=0)
#     return return_vec.tolist()

def separate_sentence(text):
    remove_chars = '[·’!"\#$%&\'()＃！（）*+,-./:;<=>?\@，：?￥★、…．＞【】［］《》？“”‘’\[\\]^_`{|}~]+'
    text = re.sub(remove_chars, "", text)

    disease_List = nltk.word_tokenize(text)

    filtered = [w for w in disease_List if(w not in stopwords.words('english'))]

    Rfiltered =nltk.pos_tag(filtered)

    filter_word = [i[0] for i in Rfiltered]
    return filter_word


def return_w2v(text):
    words_list = separate_sentence(text)
    vec = mean_vector(words_list)

    return vec


def cos_sim(a, b):
    a_norm = np.linalg.norm(a)
    b_norm = np.linalg.norm(b)
    cos = np.dot(a,b)/(a_norm * b_norm)
    return cos


def return_center(w2v_df):
    nums = len(w2v_df)
    res = np.zeros((300,))

    for i in range(nums):
        res += np.array(w2v_df[i])

    return res/nums


In [30]:
# all_similarity = []
all_sse = []
for file in tqdm(list1):
    CreationDate_list = []
    Title_list = []
    Tags_list = []
    Score_list = []
    AnswerCount_list = []
    CommentCount_list = []
    FavoriteCount_list = []
    with open(path1+file, encoding='utf-8') as f:
        lines = f.readlines()
        #lines = list(set(lines))
        if len(lines) < 2:
            all_sse.append(np.nan)
            continue
        for l in lines:
            CreationDate, Title, Tags, Score, AnswerCount, CommentCount, FavoriteCount = extract_info(l)
            CreationDate_list.append(CreationDate)
            Title_list.append(Title)
            Tags_list.append(Tags)
            Score_list.append(Score)
            AnswerCount_list.append(AnswerCount)
            CommentCount_list.append(CommentCount)
            FavoriteCount_list.append(FavoriteCount)

    new_df = pd.DataFrame({'CreationDate':[],'Title':[], 'Tags':[],
                          'Score':[],'AnswerCount':[], 'CommentCount':[],
                          'FavoriteCount':[]})
    new_df['CreationDate'] = CreationDate_list
    new_df['Title'] = Title_list
    new_df['Tags'] = Tags_list
    new_df['Score'] = Score_list
    new_df['AnswerCount'] = AnswerCount_list
    new_df['CommentCount'] = CommentCount_list
    new_df['FavoriteCount'] =FavoriteCount_list
    # print(new_df.info())



    text = new_df['Title'] + pd.Series([' ']*len(new_df)) + new_df['Tags']
    # text = text.to_list()
    w2v_df = text.apply(lambda x: return_w2v(x))
    center_vec = return_center(w2v_df)
    nums_vec = len(w2v_df)
    sum_vec = 0
    for i in range(nums_vec):
        sum_vec += ((np.array(w2v_df[i])-center_vec)**2).sum()
    # similarity_list = []
    # for i in range(len(w2v_df)):
    #     for j in range(i+1,len(w2v_df)):
    #         similarity_list.append(cos_sim(w2v_df[i],w2v_df[j]))
    #
    # all_similarity.append(np.mean(similarity_list))
    all_sse.append(sum_vec/nums_vec)


100%|██████████| 21622/21622 [1:53:03<00:00,  3.19it/s]   


In [33]:
all_sse = pd.Series(all_sse)
all_sse.to_csv('./all_sse.txt', index=False)

In [34]:
len(all_sse)

21622